# Project Setup – Knee Osteoarthritis KL Grading

**Project:** AI-Driven Detection and Grading of Knee Osteoarthritis  
**Objective:** Prepare the environment and define the experimental setup for knee X-ray–based KL grading.

This notebook focuses on:
- Environment setup
- Project configuration
- Dataset planning (final dataset selection)
- Methodology outline
- Phase 2 status summary (baseline results)

No heavy training is performed here (training is done in a separate baseline notebook).


In [1]:
# Basic environment check
import sys
print("Python version:", sys.version)

In [2]:
# Core scientific libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Deep learning libraries
import torch
import torchvision

print("Libraries imported successfully")

In [3]:
# Reproducibility
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()
print("Random seed set")

In [4]:
# Project configuration (aligned to Phase 2 baseline)
CONFIG = {
    "image_size": (224, 224),
    "batch_size": 32,
    "num_classes": 5,  # KL grades 0–4
    "learning_rate": 1e-4,
    "epochs": 5,  # baseline run (Phase 2)
    "model_backbone": "ResNet18 (ImageNet pretrained)",
    "loss": "CrossEntropy (class-weighted)",
    "metrics": ["accuracy", "macro_f1"],
}

CONFIG

## Dataset Planning (Updated After Phase 2)

### Final Selected Dataset (Phase 2)
- **Dataset:** Public Knee Osteoarthritis X-ray dataset with **Kellgren–Lawrence (KL) grades 0–4**
- **Structure:** Folder-based labeling with predefined splits:
  - `train/0..4`, `val/0..4`, `test/0..4` (plus optional `auto_test/`)
- **Reason:** OAI/MOST access can be delayed; this dataset is public and supports reproducible experimentation.

### Local Path Convention
Expected local structure (not committed to GitHub):
```
data/raw/knee-osteoarthritis-dataset-with-severity/
  train/0..4/
  val/0..4/
  test/0..4/
  auto_test/
```

### Key Observation
- Strong **class imbalance**, especially **KL 4** being underrepresented.


In [5]:
# Optional sanity-check cell (safe to run locally or in Colab if path exists)
from pathlib import Path

DATASET_DIR = Path("../data/raw/knee-osteoarthritis-dataset-with-severity")
print("Dataset exists:", DATASET_DIR.exists())

if DATASET_DIR.exists():
    print("Splits found:", [p.name for p in DATASET_DIR.iterdir() if p.is_dir()])

    def count_images_per_class(split_dir):
        counts = {}
        for class_dir in sorted(split_dir.iterdir()):
            if class_dir.is_dir():
                counts[class_dir.name] = len(list(class_dir.glob("*.png")))
        return counts

    for split in ["train", "val", "test"]:
        split_path = DATASET_DIR / split
        if split_path.exists():
            print(f"\n{split.upper()} counts:")
            print(count_images_per_class(split_path))


## Methodology Outline

- Use transfer learning with a pre-trained CNN backbone (baseline: **ResNet-18**)
- Replace final classification layer for 5-class KL grading (0–4)
- Apply medically appropriate augmentation (small rotations, flips, normalization)
- Address class imbalance using class-weighted loss
- Evaluate using accuracy + macro F1-score
- Use Grad-CAM for interpretability (planned in next phase)


## Phase 2 Status Summary (Baseline Results)

A baseline transfer learning experiment was implemented and evaluated:
- **Model:** ResNet-18 (ImageNet pretrained)
- **Training:** 5 epochs
- **Evaluation:** test set evaluation + confusion matrix

**Results (Phase 2 baseline):**
- **Test Accuracy:** 0.598
- **Test Macro F1:** 0.629

**Observed behavior:**
- Most confusions occur between adjacent KL grades (e.g., KL1↔KL2, KL3↔KL4), which is expected due to subtle radiographic differences.


## Next Steps (Phase 3)

- Add Grad-CAM visualizations for interpretability
- Explore stronger backbones (e.g., DenseNet-121)
- Improve imbalance handling (sampling strategies, focal loss, augmentation tuning)
- More rigorous reporting (per-class precision/recall/F1)
